In [35]:
import numpy as np
import pandas as pd
import plotly.express as px

In [36]:
class Objective:
    def __init__(self, x0: float, L: float) -> None:
        self.x0 = x0
        self.L = L
        
    def __call__(self, x: float, theta: float) -> float:
        return np.abs(x - self.x0) + (self.L / (1 + np.exp(x * theta)))
    
    def X(self, theta: float) -> float:
        if theta == 0:
            return self.x0
        
        u = self.L*np.abs(theta) - 2
        argument = (u + np.sqrt(u**2 - 4))/ 2
        x = np.log(argument) / theta
        return x
    
def generate_thetas(theta0: float, alpha: float, num: int):
    theta_min = np.round(theta0 - alpha, 5)
    theta_max = np.round(theta0 + alpha, 5)
    i = np.round((theta_max - theta_min) / num, 10)
    thetas = np.array([np.round(theta_min + i * f, 10) for f in range(num + 1)])
    return thetas

In [37]:
x0 = -5
theta0 = -3
alpha = 5
L = np.exp(7*10)/10
J = Objective(x0, L)

num_cols = 500000
num_rows = 1000

theta_algs = generate_thetas(theta0, alpha, num_cols)
theta_advs = generate_thetas(theta0, alpha, num_rows)

x_algs = np.array([J.X(t) for t in theta_algs])
x_advs = np.array([J.X(t) for t in theta_advs])

In [38]:
J_numerator = J(x_algs, theta_advs[:, np.newaxis]) 
J_denominator = J(x_advs, theta_advs)
ratio_matrix =  J_numerator / J_denominator[:, np.newaxis]

/var/folders/22/3f6b6q3n611gxtcqrgc0wkq80000gn/T/ipykernel_29904/922536203.py:7: RuntimeWarning:

overflow encountered in exp



In [39]:
max_ratio = np.max(ratio_matrix, axis=0)
max_ratio_idx = np.argmax(ratio_matrix, axis=0)

theta_alg_idx = np.argmin(max_ratio)
theta_alg = theta_algs[theta_alg_idx]

theta_adv_idx = max_ratio_idx[theta_alg_idx]
theta_adv = theta_advs[theta_adv_idx]

min_max_ratio = ratio_matrix[theta_adv_idx][theta_alg_idx]

In [40]:
print(f'x0             | {x0}')
print(f'theta0         | {theta0}')
print(f'alpha          | {alpha}')
print(f'Lambda         | {L:.3e}')
print(f'theta Alg      | {theta_alg}')
print(f'theta Adv      | {theta_adv}')
print(f'Min Max Ratio  | {min_max_ratio}')

x0             | -5
theta0         | -2
alpha          | 5
Lambda         | 2.515e+29
theta Alg      | 0.0
theta Adv      | 3.0
Min Max Ratio  | 8.899371767628764e+27


In [22]:
im = ratio_matrix
im_x = theta_algs
im_y = theta_advs

fig = px.imshow(im,
                labels=dict(x='thetaAlg', y='thetaAdv', color='ratio'),
                x=im_x,
                y=im_y,
                color_continuous_scale=["lightblue", "blue", "yellow", "orange", "red"],
                aspect='auto',
                text_auto=True,
                )

fig.update_layout(
    title={
        'text': f'x0={x0}     theta0={theta0}     alpha={alpha}     lambda={L}', 
        'x': 0.5, 
        'xanchor': 'center', 
        'yanchor': 'top'
        }
)

fig.show()

In [7]:
di = {
    'thetaAlg': theta_algs,
    'thetaAdv': theta_advs[max_ratio_idx],
    'max ratio': max_ratio,
}
title = f'x0={x0}     theta0={theta0}     alpha={alpha}     lambda={L:.3e}'
fig = px.scatter(di, x='thetaAlg', y ='thetaAdv', color='max ratio', color_continuous_scale=["lightblue","blue","yellow","orange","red"], title=title)
fig.update_traces(marker=dict(size=3))
# fig.update_yaxes(autorange="reversed")
# fig.update_xaxes(autorange="reversed")

fig.show() 